# test_runtime_profiling

In [1]:
import time
import tvm
from tvm.contrib.utils import tempdir
from tvm.runtime.module import BenchmarkResult

In [2]:
r = BenchmarkResult([1, 2, 2, 5])
r

BenchmarkResult(min=1, mean=2.5, median=2.0, max=5, std=1.5, results=[1, 2, 2, 5])

In [3]:
tmp = tempdir()
filename = tmp.relpath("log")

In [4]:
@tvm.register_func
def my_debug(filename):
    """one call lasts for 100 ms and writes one character to a file"""
    time.sleep(0.1)
    with open(filename, "a") as fout:
        fout.write("c")

In [6]:
X = tvm.te.compute((), lambda: tvm.tir.call_packed("my_debug", filename))
s = tvm.te.create_schedule(X.op)
func = tvm.build(s, [X])

In [7]:
x = tvm.nd.empty((), dtype="int32")
ftimer = func.time_evaluator(func.entry_name, tvm.cpu(), number=1, repeat=1)
ftimer(x)

BenchmarkResult(min=0.10030457400000001, mean=0.10030457400000001, median=0.10030457400000001, max=0.10030457400000001, std=0.0, results=(0.10030457400000001,))

In [ ]:
with open(filename, "r") as fin:
    ct = len(fin.readline())
assert ct == 2

In [ ]:
ftimer = func.time_evaluator(func.entry_name, tvm.cpu(), number=1, repeat=1, min_repeat_ms=1000)
ftimer(x)
# make sure we get more than 10 calls
with open(filename, "r") as fin:
    ct = len(fin.readline())

assert ct > 10 + 2